In [ ]:
![Header](https://github.com/VarunBurde/Prague_ml/blob/main/visulization_data/header.png?raw=true)

# Multi-View 3D Reconstruction using COLMAP

This notebook demonstrates the process of 3D reconstruction from multiple images using COLMAP, a general-purpose Structure-from-Motion (SfM) and Multi-View Stereo (MVS) pipeline.

## Step 1: Download COLMAP Source Code
First, we'll download the COLMAP source code from Google Drive.

In [1]:
import gdown

url = 'https://drive.google.com/file/d/1-Q_lZ7wW7gCWTWV-FWTGEjOZPEnPG7gE/view?usp=drive_link'
output_path = 'colmap.zip'
gdown.download(url, output_path, quiet=False,fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1-Q_lZ7wW7gCWTWV-FWTGEjOZPEnPG7gE
From (redirected): https://drive.google.com/uc?id=1-Q_lZ7wW7gCWTWV-FWTGEjOZPEnPG7gE&confirm=t&uuid=9316835a-18f8-4356-9b3f-f4c08624b5ea
To: /content/colmap.zip
100%|██████████| 119M/119M [00:03<00:00, 34.7MB/s]


'colmap.zip'

## Step 2: Extract the COLMAP Source
Extract the downloaded zip file to the root directory.

In [2]:
!unzip /content/colmap.zip -d /

Archive:  /content/colmap.zip
   creating: /content/colmap/
  inflating: /content/colmap/.clang-tidy  
  inflating: /content/colmap/vcpkg.json  
   creating: /content/colmap/cmake/
  inflating: /content/colmap/cmake/FindFreeImage.cmake  
  inflating: /content/colmap/cmake/FindDependencies.cmake  
  inflating: /content/colmap/cmake/FindGlog.cmake  
  inflating: /content/colmap/cmake/colmap-config-version.cmake.in  
  inflating: /content/colmap/cmake/CMakeUninstall.cmake.in  
  inflating: /content/colmap/cmake/FindCryptoPP.cmake  
  inflating: /content/colmap/cmake/FindLZ4.cmake  
  inflating: /content/colmap/cmake/CMakeHelper.cmake  
  inflating: /content/colmap/cmake/FindMetis.cmake  
  inflating: /content/colmap/cmake/GenerateVersionDefinitions.cmake  
  inflating: /content/colmap/cmake/FindFLANN.cmake  
  inflating: /content/colmap/cmake/FindGlew.cmake  
  inflating: /content/colmap/cmake/colmap-config.cmake.in  
   creating: /content/colmap/docker/
  inflating: /content/colmap/docke

## Step 3: Install Dependencies
COLMAP requires several libraries and dependencies. This step installs them using apt-get.

In [3]:
!sudo apt-get install \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libgmock-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libboost-program-options-dev is already the newest version (1.74.0.3ubuntu7).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.74.0.3ubuntu7).
libboost-system-dev set to manually installed.
libboost-graph-dev is already the newest version (1.74.0.3ubuntu7).
libboost-graph-dev set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.12).
libsqlite3-dev is already the newest version (3.37.2-2ubuntu0.3).
libsqlite3-dev set to manually installed.
The following additional packages will be installed:
  googletest libamd2 libbtf1 libcamd2 libccolamd2 libceres2 libcholmod3
  libcolamd2 libcxsparse3 libegl-dev libevdev2 libflann1.9 libfreeimage3
  libgflags-dev libgflags2.2 libgl-dev libglu1-mesa

## Step 4: Install COLMAP
Navigate to the build directory and install COLMAP using Ninja.

In [4]:
%cd /content/colmap/build
! ninja install

/content/colmap/build
[0/1] Install the project...
-- Install configuration: "Release"
-- Installing: /usr/local/share/applications/COLMAP.desktop
-- Installing: /usr/local/lib/libcolmap_controllers.a
-- Installing: /usr/local/lib/libcolmap_estimators.a
-- Installing: /usr/local/lib/libcolmap_exe.a
-- Installing: /usr/local/lib/libcolmap_feature_types.a
-- Installing: /usr/local/lib/libcolmap_feature.a
-- Installing: /usr/local/lib/libcolmap_geometry.a
-- Installing: /usr/local/lib/libcolmap_image.a
-- Installing: /usr/local/lib/libcolmap_math.a
-- Installing: /usr/local/lib/libcolmap_mvs.a
-- Installing: /usr/local/lib/libcolmap_optim.a
-- Installing: /usr/local/lib/libcolmap_retrieval.a
-- Installing: /usr/local/lib/libcolmap_scene.a
-- Installing: /usr/local/lib/libcolmap_sensor.a
-- Installing: /usr/local/lib/libcolmap_sfm.a
-- Installing: /usr/local/lib/libcolmap_util.a
-- Installing: /usr/local/lib/libcolmap_poisson_recon.a
-- Installing: /usr/local/lib/libcolmap_vlfeat.a
-- Inst

## Step 5: Clone the Dataset Repository
Clone the repository containing the dataset we'll use for reconstruction.

In [5]:
%cd /content/
! git clone https://github.com/VarunBurde/Prague_ml.git

/content
Cloning into 'Prague_ml'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 242 (delta 13), reused 22 (delta 7), pack-reused 213 (from 4)
Receiving objects: 100% (242/242), 235.11 MiB | 15.66 MiB/s, done.
Resolving deltas: 100% (49/49), done.


## Step 6: Prepare the Dataset
Set up the reconstruction directory and create a COLMAP database. COLMAP uses this SQLite database to store feature information about each image.

In [6]:
import os
import shutil

dataset_path = "/content/Prague_ml/dataset/ricaip"
output_dir = "/content/reconstruction"

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
shutil.copytree(dataset_path, output_dir)

database_path = os.path.join(output_dir, "database.db")
image_path = os.path.join(output_dir, "images")
!colmap database_creator --database_path {database_path}


## Step 7: Extract Features
Extract SIFT features from each image. We specify OPENCV camera model and indicate all images are from a single camera.

In [7]:
!colmap feature_extractor \
    --database_path {database_path} \
    --image_path {image_path} \
    --ImageReader.camera_model OPENCV \
    --ImageReader.single_camera 1

I0425 18:26:12.121347  1981 misc.cc:44] 
Feature extraction
I0425 18:26:12.122371  1984 sift.cc:721] Creating SIFT GPU feature extractor
I0425 18:26:12.593905  1985 feature_extraction.cc:258] Processed file [1/13]
I0425 18:26:12.593968  1985 feature_extraction.cc:261]   Name:            image_1745412812.png
I0425 18:26:12.593978  1985 feature_extraction.cc:270]   Dimensions:      2448 x 2048
I0425 18:26:12.593987  1985 feature_extraction.cc:273]   Camera:          #1 - OPENCV
I0425 18:26:12.593995  1985 feature_extraction.cc:276]   Focal Length:    2937.60px
I0425 18:26:12.594013  1985 feature_extraction.cc:280]   Features:        8841
I0425 18:26:12.728612  1985 feature_extraction.cc:258] Processed file [2/13]
I0425 18:26:12.728652  1985 feature_extraction.cc:261]   Name:            image_1745412817.png
I0425 18:26:12.728662  1985 feature_extraction.cc:270]   Dimensions:      2448 x 2048
I0425 18:26:12.728670  1985 feature_extraction.cc:273]   Camera:          #1 - OPENCV
I0425 18:26:

## Step 8: Match Features
Match features between images using exhaustive matching. This compares every image against every other image to find correspondences.

In [8]:
!colmap exhaustive_matcher \
    --database_path {database_path}


I0425 18:26:14.820703  2004 misc.cc:44] 
Feature matching
I0425 18:26:14.821368  2005 sift.cc:1426] Creating SIFT GPU feature matcher
I0425 18:26:14.907723  2004 pairing.cc:168] Generating exhaustive image pairs...
I0425 18:26:14.907759  2004 pairing.cc:201] Matching block [1/1, 1/1]
I0425 18:26:15.989902  2004 feature_matching.cc:46] in 1.082s
I0425 18:26:15.992734  2004 timer.cc:91] Elapsed time: 0.020 [minutes]


## Step 9: Sparse Reconstruction
Perform the sparse reconstruction (Structure-from-Motion) to estimate camera poses and a sparse 3D point cloud.

In [9]:
sparse_folder_path = os.path.join(output_dir, "sparse")
if os.path.exists(sparse_folder_path):
    shutil.rmtree(sparse_folder_path)

os.makedirs(sparse_folder_path)

!colmap mapper \
    --database_path {database_path}  \
    --image_path {image_path} \
    --output_path {sparse_folder_path} \


I0425 18:26:16.111056  2013 incremental_pipeline.cc:251] Loading database
I0425 18:26:16.112619  2013 database_cache.cc:66] Loading cameras...
I0425 18:26:16.112664  2013 database_cache.cc:76]  1 in 0.000s
I0425 18:26:16.112681  2013 database_cache.cc:84] Loading matches...
I0425 18:26:16.116205  2013 database_cache.cc:89]  78 in 0.004s
I0425 18:26:16.116226  2013 database_cache.cc:105] Loading images...
I0425 18:26:16.123093  2013 database_cache.cc:153]  13 in 0.007s (connected 13)
I0425 18:26:16.123119  2013 database_cache.cc:164] Loading pose priors...
I0425 18:26:16.123210  2013 database_cache.cc:175]  0 in 0.000s
I0425 18:26:16.123221  2013 database_cache.cc:184] Building correspondence graph...
I0425 18:26:16.169994  2013 database_cache.cc:210]  in 0.047s (ignored 0)
I0425 18:26:16.170040  2013 timer.cc:91] Elapsed time: 0.001 [minutes]
I0425 18:26:16.171490  2013 incremental_pipeline.cc:297] Finding good initial image pair
I0425 18:26:16.367835  2013 incremental_pipeline.cc:321]

## Step 10: Image Undistortion
Undistort the images using the camera parameters estimated during sparse reconstruction. This prepares the images for dense reconstruction.

In [10]:
dense_folder_path = os.path.join(output_dir, "dense")
if os.path.exists(dense_folder_path):
    shutil.rmtree(dense_folder_path)

os.makedirs(dense_folder_path)

!colmap image_undistorter \
    --image_path {image_path} \
    --input_path {os.path.join(output_dir, "sparse/0")} \
    --output_path {dense_folder_path} \
    --output_type COLMAP \
    --max_image_size 2000

I0425 18:27:19.728149  2303 misc.cc:44] 
Reading reconstruction
I0425 18:27:19.762434  2303 image.cc:346] => Reconstruction with 13 images and 10795 points
I0425 18:27:19.762475  2303 misc.cc:44] 
Image undistortion
I0425 18:27:19.763192  2303 undistortion.cc:215] Undistorting image [1/13]
I0425 18:27:24.345608  2303 undistortion.cc:215] Undistorting image [2/13]
I0425 18:27:24.375780  2303 undistortion.cc:215] Undistorting image [3/13]
I0425 18:27:30.240985  2303 undistortion.cc:215] Undistorting image [4/13]
I0425 18:27:30.241019  2303 undistortion.cc:215] Undistorting image [5/13]
I0425 18:27:34.528232  2303 undistortion.cc:215] Undistorting image [6/13]
I0425 18:27:34.676026  2303 undistortion.cc:215] Undistorting image [7/13]
I0425 18:27:39.153352  2303 undistortion.cc:215] Undistorting image [8/13]
I0425 18:27:39.153390  2303 undistortion.cc:215] Undistorting image [9/13]
I0425 18:27:44.733055  2303 undistortion.cc:215] Undistorting image [10/13]
I0425 18:27:45.027704  2303 undis

## Step 11: Dense Stereo Matching
Perform multi-view stereo to generate depth maps for each image using PatchMatch stereo with geometric consistency.

In [11]:
!colmap patch_match_stereo \
    --workspace_path {dense_folder_path} \
    --workspace_format COLMAP \
    --PatchMatchStereo.geom_consistency true


I0425 18:27:52.797602  2448 patch_match.cc:205] Reading workspace...
I0425 18:27:52.850394  2448 patch_match.cc:235] Reading configuration...
I0425 18:27:52.887002  2448 patch_match.cc:366] Configuration has 13 problems...
I0425 18:27:52.931596  2451 misc.cc:44] 
Processing view 1 / 13 for image_1745412812.png
I0425 18:27:52.931710  2451 patch_match.cc:461] Reading inputs...
I0425 18:27:55.445196  2451 misc.cc:51] 
PatchMatch::Problem
-------------------
I0425 18:27:55.445233  2451 patch_match.cc:54] ref_image_idx: 0
I0425 18:27:55.445245  2451 patch_match.cc:55] src_image_idxs: 
I0425 18:27:55.445252  2451 patch_match.cc:58] 10 
I0425 18:27:55.445268  2451 patch_match.cc:58] 11 
I0425 18:27:55.445276  2451 patch_match.cc:58] 8 
I0425 18:27:55.445282  2451 patch_match.cc:58] 9 
I0425 18:27:55.445288  2451 patch_match.cc:58] 7 
I0425 18:27:55.445295  2451 patch_match.cc:58] 12 
I0425 18:27:55.445302  2451 patch_match.cc:58] 3 
I0425 18:27:55.445308  2451 patch_match.cc:58] 5 
I0425 18:2

## Step 12: Stereo Fusion
Fuse the depth maps to create a dense point cloud (PLY file).

In [12]:
fused_ply_path = os.path.join(dense_folder_path, "fused.ply")

!colmap stereo_fusion \
    --workspace_path {dense_folder_path} \
    --workspace_format COLMAP \
    --input_type geometric \
    --output_path {fused_ply_path}

I0425 18:38:58.224112  5259 misc.cc:51] 
StereoFusion::Options
---------------------
I0425 18:38:58.224867  5259 fusion.cc:76] mask_path: 
I0425 18:38:58.224934  5259 fusion.cc:77] max_image_size: -1
I0425 18:38:58.224951  5259 fusion.cc:78] min_num_pixels: 5
I0425 18:38:58.224962  5259 fusion.cc:79] max_num_pixels: 10000
I0425 18:38:58.224972  5259 fusion.cc:80] max_traversal_depth: 100
I0425 18:38:58.224984  5259 fusion.cc:81] max_reproj_error: 2
I0425 18:38:58.225028  5259 fusion.cc:82] max_depth_error: 0.01
I0425 18:38:58.225044  5259 fusion.cc:83] max_normal_error: 10
I0425 18:38:58.225059  5259 fusion.cc:84] check_num_images: 50
I0425 18:38:58.225069  5259 fusion.cc:85] use_cache: 0
I0425 18:38:58.225081  5259 fusion.cc:86] cache_size: 32
I0425 18:38:58.225095  5259 fusion.cc:89] bbox_min: -3.40282e+38 -3.40282e+38 -3.40282e+38
I0425 18:38:58.225136  5259 fusion.cc:90] bbox_max: 3.40282e+38 3.40282e+38 3.40282e+38
I0425 18:38:58.225168  5259 fusion.cc:140] Reading workspace...
I0

## Step 13: Surface Reconstruction
Create a 3D mesh from the dense point cloud using Poisson surface reconstruction.

In [13]:
output_ply_path = os.path.join(dense_folder_path, "meshed-poisson.ply")

!colmap poisson_mesher \
    --input_path {fused_ply_path} \
    --output_path {output_ply_path}

Value Range: [3.435054,11.656252]


## Results

The reconstruction process has created several outputs:

1. **Sparse Point Cloud**: Located in `/content/reconstruction/sparse/0`
2. **Dense Point Cloud**: Located at `/content/reconstruction/dense/fused.ply`
3. **3D Mesh**: Located at `/content/reconstruction/dense/meshed-poisson.ply`

You can download these files to visualize them in software like MeshLab or CloudCompare.

![Footer](https://github.com/VarunBurde/Prague_ml/blob/main/visulization_data/footer.png?raw=true)